In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
import caffe
from google.protobuf import text_format

# configure plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (3,3)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Load the pretrained model

The model has been pretrained using Caffe framework during 40 epochs (20.000 iterations of training with a batchsize of 100 samples and 50.000 MNIST images).

In [2]:
caffe.set_mode_cpu()
model_path = 'models'
model_name = 'small'
path = os.path.join(model_path,model_name)

net_fn   = os.path.join(model_path, model_name, 'deploy.prototxt')
param_fn = os.path.join(model_path, model_name, 'small_iter_20000.caffemodel')

# Patching model to be able to compute gradients.
# Note that you can also manually add "force_backward: true" line to "deploy.prototxt".
model = caffe.io.caffe_pb2.NetParameter()
text_format.Merge(open(net_fn).read(), model)
model.force_backward = True
open('tmp.prototxt', 'w').write(str(model))

net = caffe.Classifier('tmp.prototxt', param_fn, caffe.TEST)

# Summary

Here we print a summary of the network's architecture: 
* the layers
* the feature maps at each layer with their corresponding size
* the parameters with their minimum and maximi value per layer (without the biases).

In [3]:
def network_summary(net):
    """
    Returns a string with a summary of the specified network
    """
    string = "Architecture of {} network\n---------------------".format(model_name)
    for layer in net.layers:
        string += "\n" + layer.type

    string += "\n\nFeature map sizes (Blobs)\n-------------------------"
    for layer in net.blobs.keys():
        string += "\n{} = \t{}".format(layer, net.blobs[layer].data.shape)

    string += "\n\nParameter shape\n---------------"
    for layer in net.params.keys():
        string += "\n{} = {} \t(min,max) = ({:.2f},{:.2f})".format(
                layer, net.params[layer][0].data.shape,
                net.params[layer][0].data.min(), 
                net.params[layer][0].data.max())
    string += "\n"
    return string

def print_network_summary(net):
    """
    Prints the summary of the specified network
    """
    print network_summary(net)
    
print_network_summary(net)

Architecture of small network
---------------------
Dropout
Convolution
ReLU
Pooling
Convolution
ReLU
InnerProduct
Softmax

Feature map sizes (Blobs)
-------------------------
data = 	(1, 1, 24, 24)
conv1 = 	(1, 9, 20, 20)
pool1 = 	(1, 9, 10, 10)
conv2 = 	(1, 16, 6, 6)
ip1 = 	(1, 10)
prob = 	(1, 10)

Parameter shape
---------------
conv1 = (9, 1, 5, 5) 	(min,max) = (-0.81,1.45)
conv2 = (16, 9, 5, 5) 	(min,max) = (-0.43,0.49)
ip1 = (10, 576) 	(min,max) = (-0.46,0.48)

